**Students**  
Brendan Hendricks  
Rizvee Ahmed  
Pranit Yadav  
Shahmir Javed  
Nathan Arimilli  
Sreejony Sengupta

In [ ]:
!pip install gensim

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium beautifulsoup4 pandas fake-useragent requests lxml
!python -m spacy download en_core_web_md
!pip install gensim
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [81.0 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,371 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,803 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,609 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packa

Task A: Extract 8-10k reviews. However, many reviews may not have any text and will therefore be
discarded. So be prepared to scrape a lot of pages of reviews per beer. Beeradvocate.com requires you
to log in to read reviews beyond the first page.

WebScraper

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import csv
import re
import time
import tempfile
from urllib.parse import urljoin
from fake_useragent import UserAgent

In [ ]:


BASE_URL = "https://www.beeradvocate.com"
TOP_BEERS_URL = "https://www.beeradvocate.com/beer/top-rated/"
CSV_FILENAME = 'beeradvocate_reviews.csv'
PRODUCTS_FILENAME = 'beer_products_list.csv'
MAX_BEERS = 500  # Increased to ensure we get enough
TARGET_QUALITY_REVIEWS = 10000  # Target number of actual quality reviews
MIN_REVIEW_LENGTH = 50  # Minimum characters for quality review
MAX_PAGES_PER_BEER = 8  # Pages to scrape per beer

# Login credentials
BA_USERNAME = "toadlover4"
BA_PASSWORD = "RizveeBrendan"

def setup_chrome_driver():
    chrome_options = Options()

    # Essential flags for Colab/Jupyter
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')

    # Create unique temp directory to avoid conflicts
    temp_dir = tempfile.mkdtemp()
    chrome_options.add_argument(f'--user-data-dir={temp_dir}')

    # Anti-detection
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)

    ua = UserAgent()
    chrome_options.add_argument(f'--user-agent={ua.random}')

    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        print("✓ Chrome driver setup successful")
        return driver

    except Exception as e:
        print(f"Chrome driver setup failed: {e}")
        print("Trying minimal fallback setup...")

        # Minimal fallback
        chrome_options_simple = Options()
        chrome_options_simple.add_argument('--headless')
        chrome_options_simple.add_argument('--no-sandbox')
        chrome_options_simple.add_argument('--disable-dev-shm-usage')

        temp_dir2 = tempfile.mkdtemp()
        chrome_options_simple.add_argument(f'--user-data-dir={temp_dir2}')

        driver = webdriver.Chrome(options=chrome_options_simple)
        print("✓ Fallback Chrome driver setup successful")
        return driver

def login_to_beeradvocate(driver):
    """Login to BeerAdvocate to access additional review pages"""
    try:
        print("Logging into BeerAdvocate...")
        driver.get("https://www.beeradvocate.com/community/login/")
        time.sleep(2)

        # Find and fill username
        try:
            username_field = driver.find_element(By.NAME, "login")
            username_field.clear()
            username_field.send_keys(BA_USERNAME)
        except:
            print("Could not find username field")
            return False

        # Find and fill password
        try:
            password_field = driver.find_element(By.NAME, "password")
            password_field.clear()
            password_field.send_keys(BA_PASSWORD)
        except:
            print("Could not find password field")
            return False

        # Submit form
        try:
            login_button = driver.find_element(By.CSS_SELECTOR, "input[type='submit']")
            login_button.click()
            time.sleep(3)
        except:
            print("Could not find login button")
            return False

        # Check if login was successful
        current_url = driver.current_url
        page_source = driver.page_source.lower()

        if "logout" in page_source or "welcome back" in page_source or "community" in current_url:
            print("✓ Successfully logged in!")
            return True
        else:
            print("✗ Login may have failed - continuing anyway")
            return True  # Continue even if unsure

    except Exception as e:
        print(f"Login error: {e} - continuing without login")
        return True  # Continue anyway

def init_csv_files():
    with open(CSV_FILENAME, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['product_name', 'product_review', 'user_rating'])

    with open(PRODUCTS_FILENAME, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['beer_name', 'url'])

    print("CSV files initialized")

def extract_beer_list():
    print("Fetching beer list...")

    headers = {
        'User-Agent': UserAgent().random,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
    }

    try:
        response = requests.get(TOP_BEERS_URL, headers=headers, timeout=10)
        response.raise_for_status()
        print(f"Got page: {response.status_code}")

        soup = BeautifulSoup(response.content, 'html.parser')
        beer_links = soup.find_all('a', href=re.compile(r'/beer/profile/\d+/\d+'))

        if not beer_links:
            beer_links = soup.find_all('a', href=re.compile(r'/beer/'))
            beer_links = [link for link in beer_links if 'profile' in link.get('href', '')]

        print(f"Found {len(beer_links)} beer links")

        beers = []
        seen_urls = set()

        for link in beer_links[:MAX_BEERS]:
            href = link.get('href')
            if not href:
                continue

            beer_url = urljoin(BASE_URL, href)
            if beer_url in seen_urls:
                continue
            seen_urls.add(beer_url)

            beer_name = link.text.strip()
            if not beer_name or len(beer_name) < 3:
                continue

            beers.append({
                'beer_name': beer_name,
                'url': beer_url
            })

        print(f"Collected {len(beers)} beers")
        return beers

    except Exception as e:
        print(f"Error: {e}")
        return []

def extract_reviews_from_beer_multipage(driver, beer_info):
    """Extract quality reviews from multiple pages of a beer - ONLY COUNT QUALITY ONES"""
    beer_url = beer_info['url']
    beer_name = beer_info['beer_name']
    quality_reviews = []  # Only store reviews that pass quality filter

    try:
        # Get base beer page first
        driver.get(beer_url)
        time.sleep(2)

        # Try to find reviews section or construct URL
        base_reviews_url = beer_url

        # Look for reviews link patterns
        try:
            # Common BeerAdvocate review URL patterns
            match = re.search(r'/beer/profile/(\d+)/(\d+)', beer_url)
            if match:
                brewery_id, beer_id = match.groups()
                base_reviews_url = f"https://www.beeradvocate.com/beer/profile/{brewery_id}/{beer_id}/"
        except:
            pass

        # Scrape multiple pages
        for page in range(MAX_PAGES_PER_BEER):
            try:
                # Construct page URL - BeerAdvocate uses ?start= parameter
                if page == 0:
                    page_url = base_reviews_url
                else:
                    page_url = f"{base_reviews_url}?start={25 * page}"

                driver.get(page_url)
                time.sleep(1.5)

                # Extract reviews from current page
                page_quality_reviews = extract_quality_reviews_from_current_page(driver, beer_name)

                if not page_quality_reviews:
                    print(f"     No quality reviews on page {page+1}, stopping")
                    break

                quality_reviews.extend(page_quality_reviews)
                print(f"     Page {page+1}: +{len(page_quality_reviews)} quality reviews")

                # Stop if we have enough from this beer
                if len(quality_reviews) >= 80:  # Reasonable limit per beer
                    break

            except Exception as e:
                print(f"     Error on page {page+1}: {e}")
                break

        return quality_reviews

    except Exception as e:
        print(f"   Error: {e}")
        return []

def extract_quality_reviews_from_current_page(driver, beer_name):
    """Extract ONLY quality reviews from current page - strict filtering"""
    quality_reviews = []

    try:
        # BeerAdvocate-specific selectors
        review_selectors = [
            'div[id^="rating_"]',  # Primary: BeerAdvocate uses rating_XXXXX
            'tr[id^="rating_"]',   # Alternative table format
            'div.review-content',  # Sometimes uses this class
            'tr[bgcolor]',         # Colored table rows
            'table tr'             # Fallback
        ]

        review_elements = []
        for selector in review_selectors:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            if len(elements) >= 5:  # Found reasonable number
                review_elements = elements
                break

        # Process each element
        for element in review_elements[:50]:  # Limit per page
            try:
                review_data = extract_single_quality_review(element, beer_name)
                if review_data:  # Only add if it passes quality check
                    quality_reviews.append(review_data)
            except:
                continue

        return quality_reviews

    except Exception as e:
        return []

def extract_single_quality_review(review_elem, beer_name):
    """Extract single review with STRICT quality filtering"""
    try:
        # Get text content
        full_text = ""
        if hasattr(review_elem, 'text'):
            full_text = review_elem.text
        else:
            full_text = review_elem.get_attribute('textContent') or ""

        if len(full_text.strip()) < MIN_REVIEW_LENGTH:
            return None  # Too short

        # Extract rating
        rating = extract_rating_from_text(full_text)

        # Extract and clean review text
        review_text = extract_clean_review_text(full_text)

        # QUALITY FILTERS - all must pass
        if not review_text or len(review_text) < MIN_REVIEW_LENGTH:
            return None

        if not has_beer_content(review_text):
            return None

        if len(review_text.split()) < 10:  # At least 10 words
            return None

        # Must have meaningful content
        if is_meaningless_text(review_text):
            return None

        return {
            'product_name': beer_name,
            'product_review': review_text,
            'user_rating': rating if rating else 0
        }

    except Exception as e:
        return None

def extract_rating_from_text(text):
    """Extract rating with comprehensive patterns"""
    rating_patterns = [
        r'(\d+\.?\d*)\s*/\s*5',
        r'(\d+\.?\d*)\s*/\s*10',
        r'(\d+\.?\d*)\s*/\s*100',
        r'Overall:\s*(\d+\.?\d*)',
        r'Score:\s*(\d+\.?\d*)',
        r'Rating:\s*(\d+\.?\d*)',
        r'look:\s*(\d+\.?\d*)',
        r'smell:\s*(\d+\.?\d*)',
        r'taste:\s*(\d+\.?\d*)',
        r'feel:\s*(\d+\.?\d*)',
        r'overall:\s*(\d+\.?\d*)'
    ]

    for pattern in rating_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            try:
                raw_rating = float(match.group(1))

                # Convert to 5-point scale
                if '/5' in pattern:
                    return min(raw_rating, 5.0)
                elif '/10' in pattern:
                    return min(raw_rating / 2, 5.0)
                elif '/100' in pattern:
                    return min(raw_rating / 20, 5.0)
                else:
                    # Guess based on value
                    if raw_rating <= 5:
                        return raw_rating
                    elif raw_rating <= 10:
                        return raw_rating / 2
                    elif raw_rating <= 100:
                        return raw_rating / 20

            except ValueError:
                continue

    return None

def extract_clean_review_text(full_text):
    """Extract clean review text, removing metadata"""

    # Remove metadata patterns
    text = re.sub(r'Reviewed by.*?on.*?\d{4}', '', full_text, flags=re.IGNORECASE)
    text = re.sub(r'Location:.*?Date:.*?\d{4}', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\d+\.?\d*\s*/\s*\d+', ' ', text)  # Remove ratings
    text = re.sub(r'(look|smell|taste|feel|overall):\s*\d+\.?\d*', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'Serving type:.*?Container:.*?', '', text, flags=re.IGNORECASE)

    # Clean up whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    # Find the longest coherent part
    sentences = re.split(r'[.!?]+', text)
    best_part = ""

    for sentence in sentences:
        if len(sentence.strip()) > len(best_part) and has_beer_content(sentence):
            best_part = sentence.strip()

    if not best_part:
        best_part = text

    return best_part[:800]  # Reasonable length limit

def has_beer_content(text):
    """Check if text has actual beer review content"""
    beer_keywords = [
        'taste', 'flavor', 'aroma', 'smell', 'appearance', 'look', 'beer', 'hop', 'hops',
        'malt', 'bitter', 'sweet', 'drink', 'pour', 'head', 'foam', 'alcohol', 'finish',
        'aftertaste', 'body', 'mouthfeel', 'carbonation', 'crisp', 'smooth', 'dry',
        'light', 'dark', 'amber', 'golden', 'brown', 'wheat', 'barley', 'yeast', 'brew'
    ]

    text_lower = text.lower()
    beer_word_count = sum(1 for keyword in beer_keywords if keyword in text_lower)

    return beer_word_count >= 2

def is_meaningless_text(text):
    """Filter out meaningless/spam text"""
    meaningless_patterns = [
        r'^[\d\s\.\-/]+$',  # Just numbers and symbols
        r'^[A-Za-z\s]{1,20}$',  # Too short and generic
        r'test|spam|asdf|qwerty',  # Common spam
    ]

    for pattern in meaningless_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return True

    return False

def save_reviews_to_csv(reviews):
    if not reviews:
        return

    with open(CSV_FILENAME, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=[
            'product_name', 'product_review', 'user_rating'
        ])
        for review in reviews:
            writer.writerow(review)

def save_beers_to_csv(beers):
    with open(PRODUCTS_FILENAME, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=['beer_name', 'url'])
        for beer in beers:
            writer.writerow(beer)

# MAIN EXECUTION
init_csv_files()
print("Setup complete")

# Get beer list
print("Getting beer list...")
beer_list = extract_beer_list()

if not beer_list:
    print("No beers found")
    exit()

print(f"Got {len(beer_list)} beers")
save_beers_to_csv(beer_list)

# Setup driver and login
driver = setup_chrome_driver()
logged_in = login_to_beeradvocate(driver)

# Scrape reviews - ONLY COUNT QUALITY ONES
print(f"\nScraping quality reviews (target: {TARGET_QUALITY_REVIEWS})...")
print(f"Minimum review length: {MIN_REVIEW_LENGTH} characters")

total_quality_reviews = 0

try:
    for i, beer in enumerate(beer_list):
        if total_quality_reviews >= TARGET_QUALITY_REVIEWS:
            print(f"🎯 TARGET REACHED! Got {total_quality_reviews} quality reviews")
            break

        try:
            remaining = TARGET_QUALITY_REVIEWS - total_quality_reviews
            print(f"[{i+1}/{len(beer_list)}] {beer['beer_name'][:45]}... (need {remaining} more)")

            # Get quality reviews from this beer (multiple pages)
            quality_reviews = extract_reviews_from_beer_multipage(driver, beer)

            if quality_reviews:
                save_reviews_to_csv(quality_reviews)
                total_quality_reviews += len(quality_reviews)
                print(f"   ✅ +{len(quality_reviews)} quality reviews (Total: {total_quality_reviews}/{TARGET_QUALITY_REVIEWS})")
            else:
                print("   ❌ No quality reviews found")

            # Progress checkpoints
            if (i + 1) % 15 == 0:
                print(f"\n📊 CHECKPOINT: {total_quality_reviews}/{TARGET_QUALITY_REVIEWS} quality reviews")

        except KeyboardInterrupt:
            print("\n⏹️ Stopped by user")
            break
        except Exception as e:
            print(f"   ❌ Error: {e}")
            continue

finally:
    driver.quit()

print(f"\n🎉 SCRAPING COMPLETE!")
print(f"Total QUALITY reviews collected: {total_quality_reviews}")

# Final analysis
try:
    df = pd.read_csv(CSV_FILENAME)
    print(f"\n📈 FINAL ANALYSIS:")
    print(f"Dataset size: {len(df)} reviews")

    if len(df) > 0:
        print(f"Average rating: {df['user_rating'].mean():.2f}/5.0")
        print(f"Average review length: {df['product_review'].str.len().mean():.0f} characters")
        print(f"Reviews with ratings: {len(df[df['user_rating'] > 0])}")
        print(f"Unique beers: {df['product_name'].nunique()}")

        print(f"\n📝 SAMPLE REVIEW:")
        sample = df.iloc[0]
        print(f"Beer: {sample['product_name']}")
        print(f"Rating: {sample['user_rating']}")
        print(f"Review: {sample['product_review'][:250]}...")

except Exception as e:
    print(f"Could not analyze final data: {e}")

print("\n✅ DONE!")

CSV files initialized
Setup complete
Getting beer list...
Fetching beer list...
Error: 403 Client Error: Forbidden for url: https://www.beeradvocate.com/beer/top-rated/
No beers found
Got 0 beers
Chrome driver setup failed: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x5580ef15afba <unknown>
#1 0x5580eebdf6d0 <unknown>
#2 0x5580eec1a475 <unknown>
#3 0x5580eec16368 <unknown>
#4 0x5580eec65280 <unknown>
#5 0x5580eec64946 <unknown>
#6 0x5580eec56c03 <unknown>
#7 0x5580eec237a8 <unknown>
#8 0x5580eec24421 <unknown>
#9 0x5580ef11fb28 <unknown>
#10 0x5580ef12387f <unknown>
#11 0x5580ef107c49 <unknown>
#12 0x5580ef124405 <unknown>
#13 0x5580ef0ed4ff <unknown>
#14 0x5580ef148258 <unknown>
#15 0x5580ef148432 <unkno

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x562f96738fba <unknown>
#1 0x562f961bd6d0 <unknown>
#2 0x562f961f8475 <unknown>
#3 0x562f961f4368 <unknown>
#4 0x562f96243280 <unknown>
#5 0x562f96242946 <unknown>
#6 0x562f96234c03 <unknown>
#7 0x562f962017a8 <unknown>
#8 0x562f96202421 <unknown>
#9 0x562f966fdb28 <unknown>
#10 0x562f9670187f <unknown>
#11 0x562f966e5c49 <unknown>
#12 0x562f96702405 <unknown>
#13 0x562f966cb4ff <unknown>
#14 0x562f96726258 <unknown>
#15 0x562f96726432 <unknown>
#16 0x562f96737fa3 <unknown>
#17 0x7d7388ef1ac3 <unknown>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Navigate to your file - it will be in /content/drive/MyDrive/
# Replace 'your_filename.csv' with your actual filename
df = pd.read_csv('/content/drive/MyDrive/beeradvocate_reviews.csv')
print(f"Loaded {len(df)} rows")

# Save it locally in Colab for your existing code to use
df.to_csv('beeradvocate_reviews.csv', index=False)
print("File ready for your Task B code")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 2875 rows
File ready for your Task B code


Task B: Assume that a customer, who will be using this recommender system, has specified 3 attributes
in a product. E.g., one website describes multiple attributes of beer (but you should choose attributes
from the actual data like you did for the first assignment)


**Chosen Attributes**: Chocolate, Dark, Coffee

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
import re
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings('ignore')

print("🍺 Task B: Truly Data-Driven Beer Recommendation System")
print("="*70)

# ================================
# STEP 1: DATA LOADING & PREPROCESSING
# ================================

try:
    print("Loading beer review data...")
    df = pd.read_csv('beeradvocate_reviews.csv')
    print(f"✓ Loaded {len(df)} reviews for {df['product_name'].nunique()} unique beers")

    # Basic validation
    if len(df) == 0:
        print("❌ Error: Dataset is empty!")
        exit(1)

    required_columns = ['product_name', 'product_review', 'user_rating']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        print(f"❌ Error: Missing required columns: {missing_columns}")
        exit(1)

except FileNotFoundError:
    print("❌ Error: 'beeradvocate_reviews.csv' not found!")
    print("Please run the scraper first to generate the data.")
    exit(1)

def clean_text(text):
    """Clean and preprocess review text for analysis"""
    if pd.isna(text):
        return ""

    text = str(text).lower()
    # Remove special characters but keep spaces
    text = re.sub(r'[^\w\s]', ' ', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Clean the data
print("Preprocessing review text...")
df['clean_review'] = df['product_review'].apply(clean_text)
df['user_rating'] = pd.to_numeric(df['user_rating'], errors='coerce').fillna(0)

# Remove very short reviews
original_count = len(df)
df = df[df['clean_review'].str.len() > 30].copy()
print(f"Removed {original_count - len(df)} short reviews")
print(f"✓ {len(df)} reviews remaining for analysis")

# ================================
# STEP 2: TRULY DATA-DRIVEN ATTRIBUTE DISCOVERY
# ================================

print(f"\n{'='*70}")
print("STEP 2: DATA-DRIVEN ATTRIBUTE DISCOVERY")
print(f"{'='*70}")

def extract_meaningful_descriptors(reviews, min_length=3, max_length=12):
    """Extract meaningful descriptive words from all reviews - NO PREDEFINED LISTS"""

    print("Extracting all unique words from reviews...")

    # Get all words from all reviews
    all_words = []
    for review in reviews:
        if isinstance(review, str):
            words = review.split()
            # Filter for reasonable word lengths
            meaningful_words = [w for w in words if min_length <= len(w) <= max_length]
            all_words.extend(meaningful_words)

    # Count frequency of all words
    word_freq = Counter(all_words)

    # Remove common stop words and non-descriptive words
    stop_words = {
    # Original stop words
    'the', 'and', 'but', 'for', 'are', 'was', 'were', 'been', 'have', 'has', 'had',
    'this', 'that', 'with', 'from', 'they', 'them', 'their', 'said', 'each', 'which',
    'what', 'where', 'when', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
    'most', 'other', 'some', 'such', 'only', 'own', 'same', 'than', 'too', 'very',
    'can', 'will', 'just', 'should', 'now', 'also', 'back', 'after', 'first', 'well',
    'way', 'even', 'new', 'want', 'because', 'good', 'great', 'bad', 'nice', 'like',
    'really', 'much', 'many', 'little', 'long', 'time', 'year', 'get', 'make', 'take',
    'come', 'could', 'would', 'there', 'here', 'then', 'than', 'these', 'those',

    # Beer review generic terms (new additions)
    'beer', 'taste', 'your', 'review', 'smell', 'look', 'overall', 'feel', 'out',
    'head', 'don', 'plus', 'need', 'impression', 'drop', 'attributes', 'thoughts',
    'you', 'not', 'one', 'may', 'into', 'its', 'about', 'see', 'know', 'think',
    'got', 'got', 'went', 'went', 'use', 'used', 'find', 'found', 'come', 'came'
}

    # Filter out stop words and get potential descriptors
    potential_descriptors = {word: count for word, count in word_freq.items()
                           if word not in stop_words and count >= 5}  # Minimum 5 occurrences

    print(f"Found {len(potential_descriptors)} potential descriptive words")

    # Further filter for words that are likely to be beer descriptors
    # Look for adjectives and beer-related terms by checking context
    beer_descriptors = {}

    print("Filtering for beer-related descriptive terms...")

    # Keywords that indicate beer-related context
    beer_context_words = {
        'beer', 'ale', 'lager', 'stout', 'porter', 'ipa', 'brew', 'brewing', 'brewed',
        'taste', 'tastes', 'tasting', 'flavor', 'flavors', 'flavored', 'aroma', 'smell',
        'appearance', 'look', 'looks', 'color', 'head', 'foam', 'carbonation',
        'mouthfeel', 'body', 'finish', 'aftertaste', 'alcohol', 'abv', 'drink', 'drinking'
    }

    # For each potential descriptor, check if it commonly appears near beer context words
    for word, count in potential_descriptors.items():
        if count < 10:  # Need reasonable frequency
            continue

        # Check if this word appears in beer-related contexts
        beer_context_count = 0
        total_appearances = 0

        for review in reviews[:1000]:  # Sample to speed up processing
            if isinstance(review, str) and word in review:
                total_appearances += 1
                review_words = review.split()

                # Check if any beer context words appear near this word
                if word in review_words:
                    word_positions = [i for i, w in enumerate(review_words) if w == word]

                    for pos in word_positions:
                        # Check words within 5 positions before and after
                        context_start = max(0, pos - 5)
                        context_end = min(len(review_words), pos + 6)
                        context_words = review_words[context_start:context_end]

                        if any(ctx_word in beer_context_words for ctx_word in context_words):
                            beer_context_count += 1
                            break

        # If word appears in beer context frequently enough, it's likely a beer descriptor
        if total_appearances > 0 and (beer_context_count / total_appearances) > 0.3:
            beer_descriptors[word] = count

    print(f"Identified {len(beer_descriptors)} beer-related descriptive terms")

    return beer_descriptors

# Extract descriptors from actual review data
print("Analyzing review text to discover descriptive attributes...")
beer_descriptors = extract_meaningful_descriptors(df['clean_review'])

# Get top attributes by frequency
top_discovered_attributes = sorted(beer_descriptors.items(), key=lambda x: x[1], reverse=True)

print(f"\nTop 20 discovered beer attributes from review data:")
for i, (attr, count) in enumerate(top_discovered_attributes[:20], 1):
    print(f"  {i:2d}. {attr:15} ({count:4d} mentions)")

# ================================
# STEP 3: LIFT ANALYSIS FOR CO-OCCURRENCE
# ================================

print(f"\n{'='*70}")
print("STEP 3: LIFT ANALYSIS FOR ATTRIBUTE CO-OCCURRENCE")
print(f"{'='*70}")

def find_attributes_in_review(review_text, attribute_list):
    """Find which attributes appear in a given review"""
    if not isinstance(review_text, str):
        return []

    words_in_review = set(review_text.split())
    found_attributes = [attr for attr in attribute_list if attr in words_in_review]
    return found_attributes

# Create attribute occurrence matrix
print("Creating attribute co-occurrence matrix...")

# Use top 30 attributes for lift analysis
top_30_attrs = [attr for attr, count in top_discovered_attributes[:30]]

# Find attributes in each review
df['found_attributes'] = df['clean_review'].apply(
    lambda x: find_attributes_in_review(x, top_30_attrs)
)

def calculate_lift_from_data(df, min_support=10):
    """Calculate lift ratios for discovered attributes"""

    # Count individual attribute frequencies
    attr_counts = Counter()
    for attr_list in df['found_attributes']:
        if isinstance(attr_list, list):
            attr_counts.update(attr_list)

    # Filter for frequent attributes
    frequent_attrs = [attr for attr, count in attr_counts.items() if count >= min_support]
    total_reviews = len(df)

    print(f"Analyzing lift for {len(frequent_attrs)} frequent attributes...")

    # Calculate lift for all pairs
    lift_results = []

    for i, attr1 in enumerate(frequent_attrs):
        for j, attr2 in enumerate(frequent_attrs):
            if i >= j:  # Avoid duplicates and self-pairs
                continue

            # Count co-occurrences
            cooccur_count = 0
            for attr_list in df['found_attributes']:
                if isinstance(attr_list, list) and attr1 in attr_list and attr2 in attr_list:
                    cooccur_count += 1

            if cooccur_count >= 3:  # Minimum co-occurrences
                # Calculate probabilities
                prob_a = attr_counts[attr1] / total_reviews
                prob_b = attr_counts[attr2] / total_reviews
                prob_ab = cooccur_count / total_reviews

                # Calculate lift
                lift_ratio = prob_ab / (prob_a * prob_b)

                lift_results.append({
                    'attr1': attr1,
                    'attr2': attr2,
                    'cooccur_count': cooccur_count,
                    'lift_ratio': lift_ratio
                })

    # Sort by lift ratio
    lift_results.sort(key=lambda x: x['lift_ratio'], reverse=True)

    return lift_results, frequent_attrs

# Calculate lift analysis
lift_results, frequent_attrs = calculate_lift_from_data(df, min_support=15)

print(f"\nTop 15 attribute pairs with highest lift ratios:")
print(f"{'Rank':<4} {'Attribute Pair':<30} {'Lift Ratio':<12} {'Co-occurrences':<15}")
print("-" * 65)

for i, result in enumerate(lift_results[:15], 1):
    pair_name = f"{result['attr1']} & {result['attr2']}"
    print(f"{i:3d}. {pair_name:<30} {result['lift_ratio']:>8.3f}     {result['cooccur_count']:>10d}")

# ================================
# STEP 4: DATA-DRIVEN ATTRIBUTE SELECTION
# ================================

print(f"\n{'='*70}")
print("STEP 4: DATA-DRIVEN CUSTOMER ATTRIBUTE SELECTION")
print(f"{'='*70}")

print("Selection Strategy:")
print("1. Use frequency analysis to find most common descriptors")
print("2. Use lift analysis to find attributes that co-occur")
print("3. Select 3 attributes that are both frequent AND have high co-occurrence")

# Strategy: Select attributes that appear in top frequency AND top lift pairs
high_frequency_attrs = set([attr for attr, count in top_discovered_attributes[:15]])
high_lift_attrs = set()

# Get attributes from top lift pairs
for result in lift_results[:10]:
    high_lift_attrs.add(result['attr1'])
    high_lift_attrs.add(result['attr2'])

# Find intersection - attributes that are both frequent and co-occur well
candidate_attrs = high_frequency_attrs.intersection(high_lift_attrs)

print(f"\nAttributes that are both frequent AND co-occur well:")
selected_candidates = []
for attr in top_discovered_attributes:
    if attr[0] in candidate_attrs:
        selected_candidates.append(attr)

for i, (attr, count) in enumerate(selected_candidates[:10], 1):
    lift_appearances = sum(1 for result in lift_results[:10]
                          if attr in [result['attr1'], result['attr2']])
    print(f"  {i:2d}. {attr:15} ({count:4d} mentions, in {lift_appearances} top lift pairs)")

# Select final 3 attributes
if len(selected_candidates) >= 3:
    selected_attributes = [attr for attr, count in selected_candidates[:3]]
else:
    # Fallback: use top frequency attributes
    selected_attributes = [attr for attr, count in top_discovered_attributes[:3]]

print(f"\n🎯 SELECTED CUSTOMER ATTRIBUTES (Data-Driven):")
for i, attr in enumerate(selected_attributes, 1):
    attr_count = dict(top_discovered_attributes).get(attr, 0)
    in_lift = any(attr in [r['attr1'], r['attr2']] for r in lift_results[:10])
    status = "✓ High frequency + High co-occurrence" if in_lift else "✓ High frequency"
    print(f"  {i}. {attr:15} ({attr_count:4d} mentions) - {status}")

print(f"\nCustomer Simulation:")
print(f"A customer says: 'I want a beer that is {selected_attributes[0]}, {selected_attributes[1]}, and {selected_attributes[2]}'")

# ================================
# STEP 5: SENTIMENT ANALYSIS
# ================================

print(f"\n{'='*70}")
print("STEP 5: SENTIMENT ANALYSIS")
print(f"{'='*70}")

def get_sentiment_score(text):
    """Calculate sentiment polarity using TextBlob"""
    try:
        blob = TextBlob(str(text))
        return blob.sentiment.polarity  # Returns -1 to 1
    except:
        return 0

print("Calculating sentiment scores for all reviews...")
df['sentiment'] = df['product_review'].apply(get_sentiment_score)

# Display sentiment distribution
positive_count = sum(df['sentiment'] > 0.1)
neutral_count = sum((df['sentiment'] >= -0.1) & (df['sentiment'] <= 0.1))
negative_count = sum(df['sentiment'] < -0.1)

print(f"Sentiment Distribution:")
print(f"  Positive (>0.1):  {positive_count:5d} reviews ({positive_count/len(df)*100:.1f}%)")
print(f"  Neutral (-0.1-0.1): {neutral_count:5d} reviews ({neutral_count/len(df)*100:.1f}%)")
print(f"  Negative (<-0.1):  {negative_count:5d} reviews ({negative_count/len(df)*100:.1f}%)")

# ================================
# STEP 6: BEER PROFILING & TF-IDF MODELING
# ================================

print(f"\n{'='*70}")
print("STEP 6: BEER PROFILING & RECOMMENDATION MODEL")
print(f"{'='*70}")

print("Creating comprehensive beer profiles...")

# Aggregate reviews by beer
beer_profiles = df.groupby('product_name').agg({
    'clean_review': lambda x: ' '.join(x),
    'user_rating': 'mean',
    'sentiment': 'mean',
    'product_review': 'count'
}).rename(columns={'product_review': 'review_count'})

# Filter beers with sufficient reviews for reliability
min_reviews = max(2, len(df) // (df['product_name'].nunique() * 4))
reliable_beers = beer_profiles[beer_profiles['review_count'] >= min_reviews]

print(f"✓ Created profiles for {len(reliable_beers)} beers with {min_reviews}+ reviews each")

# Create customer preference profile
customer_profile = ' '.join(selected_attributes * 5)  # Repeat for emphasis

# Combine customer profile with beer profiles
all_profiles = [customer_profile] + reliable_beers['clean_review'].tolist()

print("Building TF-IDF model for semantic similarity...")

# Create TF-IDF vectors
tfidf = TfidfVectorizer(
    max_features=min(1000, len(' '.join(all_profiles).split())),
    stop_words='english',
    ngram_range=(1, 2),
    min_df=1,
    max_df=0.95
)

tfidf_matrix = tfidf.fit_transform(all_profiles)

# Calculate cosine similarity
customer_vector = tfidf_matrix[0]
beer_vectors = tfidf_matrix[1:]
similarities = cosine_similarity(customer_vector, beer_vectors).flatten()

print(f"✓ Calculated similarity scores for {len(similarities)} beers")

# ================================
# STEP 7: RECOMMENDATION GENERATION
# ================================

print(f"\n{'='*70}")
print("STEP 7: GENERATING RECOMMENDATIONS")
print(f"{'='*70}")

# Create recommendations dataframe
recommendations = pd.DataFrame({
    'beer_name': reliable_beers.index,
    'cosine_similarity': similarities,
    'avg_rating': reliable_beers['user_rating'],
    'avg_sentiment': reliable_beers['sentiment'],
    'review_count': reliable_beers['review_count']
})

# Handle any NaN values
recommendations = recommendations.fillna(0)

# Calculate composite score (weighted combination)
recommendations['composite_score'] = (
    0.4 * recommendations['cosine_similarity'] +                    # 40% Content similarity
    0.3 * (recommendations['avg_rating'] / 5.0) +                  # 30% Average rating
    0.2 * ((recommendations['avg_sentiment'] + 1) / 2) +           # 20% Sentiment
    0.1 * np.log(recommendations['review_count'] + 1) / 10         # 10% Popularity
)

# Sort by composite score
recommendations = recommendations.sort_values('composite_score', ascending=False)

print("✓ Recommendations generated and ranked")

# ================================
# STEP 8: RESULTS DISPLAY
# ================================

print(f"\n{'='*80}")
print("FINAL RESULTS: BEER RECOMMENDATIONS")
print(f"{'='*80}")

print(f"Customer Request: 'I want a beer that is {', '.join(selected_attributes)}'")
print(f"These attributes were selected through data-driven analysis of {len(df)} reviews")

print(f"\nRecommendation Methodology:")
print(f"1. Discovered {len(beer_descriptors)} attributes directly from review text")
print(f"2. Selected top 3 attributes using frequency + co-occurrence analysis")
print(f"3. Used TF-IDF + cosine similarity to match customer preferences")
print(f"4. Combined similarity (40%) + rating (30%) + sentiment (20%) + popularity (10%)")

# Display Top 3 + 20 Contenders as required
print(f"\n{'='*80}")
print("TOP 3 FINAL RECOMMENDATIONS + 20 CONTENDERS")
print(f"{'='*80}")

top_23 = recommendations.head(23)

print(f"{'Rank':<4} {'Beer Name':<40} {'Similarity':<10} {'Rating':<8} {'Sentiment':<10} {'Reviews':<8} {'Final Score':<10}")
print("-" * 95)

for i, (_, row) in enumerate(top_23.iterrows(), 1):
    if i <= 3:
        rank_marker = f"🏆{i}"
    else:
        rank_marker = f"{i:2d}."

    print(f"{rank_marker:<4} {row['beer_name'][:39]:<40} {row['cosine_similarity']:.3f}      {row['avg_rating']:.2f}     {row['avg_sentiment']:.3f}      {int(row['review_count']):<8} {row['composite_score']:.3f}")

# Detailed breakdown of top 3
print(f"\n{'='*80}")
print("DETAILED ANALYSIS OF TOP 3 RECOMMENDATIONS")
print(f"{'='*80}")

for i in range(min(3, len(top_23))):
    beer = top_23.iloc[i]
    print(f"\n🏆 RECOMMENDATION #{i+1}: {beer['beer_name']}")
    print(f"   Why this beer was selected:")
    print(f"   • Content Similarity:  {beer['cosine_similarity']:.3f} (how well it matches '{', '.join(selected_attributes)}')")
    print(f"   • Average Rating:      {beer['avg_rating']:.2f}/5.0 (overall quality)")
    print(f"   • Sentiment Score:     {beer['avg_sentiment']:.3f} (reviewer satisfaction)")
    print(f"   • Review Count:        {int(beer['review_count'])} (reliability)")
    print(f"   • Final Composite Score: {beer['composite_score']:.3f}")

# Show sample reviews for top recommendation
if len(top_23) > 0:
    print(f"\n{'='*80}")
    print("SAMPLE REVIEWS FOR TOP RECOMMENDATION")
    print(f"{'='*80}")

    top_beer_name = top_23.iloc[0]['beer_name']
    sample_reviews = df[df['product_name'] == top_beer_name].head(3)

    print(f"Beer: {top_beer_name}")

    if len(sample_reviews) > 0:
        for i, (_, review) in enumerate(sample_reviews.iterrows(), 1):
            print(f"\nSample Review {i} (Rating: {review['user_rating']}/5, Sentiment: {review['sentiment']:.3f}):")
            print(f"'{review['product_review'][:200]}...'")

# Summary statistics
print(f"\n{'='*80}")
print("DATA-DRIVEN ANALYSIS SUMMARY")
print(f"{'='*80}")

print(f"Dataset Analysis:")
print(f"  • Total reviews processed: {len(df)}")
print(f"  • Unique beers analyzed: {df['product_name'].nunique()}")
print(f"  • Descriptive attributes discovered: {len(beer_descriptors)}")
print(f"  • Attribute pairs analyzed for co-occurrence: {len(lift_results)}")

print(f"\nAttribute Selection Process:")
print(f"  • Discovered attributes from actual review text (not predefined)")
print(f"  • Used frequency analysis to identify common descriptors")
print(f"  • Applied lift analysis to find attributes that co-occur")
print(f"  • Selected final 3 attributes: {selected_attributes}")

print(f"\nRecommendation Quality:")
print(f"  • Beers evaluated: {len(recommendations)}")
print(f"  • Average similarity to customer preferences: {recommendations['cosine_similarity'].mean():.3f}")
print(f"  • Top recommendation similarity: {recommendations.iloc[0]['cosine_similarity']:.3f}")

print(f"\n✅ Task B Complete: Data-driven beer recommendation system ready!")
print(f"Next: Implement Task C (word embeddings comparison)")

🍺 Task B: Truly Data-Driven Beer Recommendation System
Loading beer review data...
✓ Loaded 3034 reviews for 249 unique beers
Preprocessing review text...
Removed 4 short reviews
✓ 3030 reviews remaining for analysis

STEP 2: DATA-DRIVEN ATTRIBUTE DISCOVERY
Analyzing review text to discover descriptive attributes...
Extracting all unique words from reviews...
Found 2687 potential descriptive words
Filtering for beer-related descriptive terms...
Identified 735 beer-related descriptive terms

Top 20 discovered beer attributes from review data:
   1. chocolate       (1209 mentions)
   2. dark            (1137 mentions)
   3. coffee          ( 909 mentions)
   4. sweet           ( 872 mentions)
   5. bourbon         ( 864 mentions)
   6. vanilla         ( 849 mentions)
   7. nose            ( 795 mentions)
   8. notes           ( 771 mentions)
   9. light           ( 755 mentions)
  10. black           ( 733 mentions)
  11. pours           ( 714 mentions)
  12. aroma           ( 649 mentio

For **Task B**, we scraped and preprocessed over 3K BeerAdvocate reviews, discarding no-text and very short entries. Reviews were aggregated by beer into a single document per product, and we computed each beer’s average rating, average sentiment, and review count.

We used a data-driven attribute discovery step (frequency + co-occurrence lift), the three customer attributes selected from the corpus were: **chocolate**, **dark**, and **coffee**.

We then applied the bag of words model and constructed a TF-IDF matrix on the beer review documents, represented the user's attributes (chocolate, dark, coffee) within the same TF-IDF space, and lastly computed a cosine similarity between the query vector and each beer vector.

Additionally, we built out a composite score- which is a weighted combination of the different signals. We combined cosine similarity with average rating, sentiment, and review count into the composite score. This ensured that our recommendations were not just the beers that mentioned the right/named attributes, but also those that were well-reviewed, positively described, and overall widely sampled.

The composite score was calculated as:
$$
\text{Score} = 0.40 \cdot \text{Cosine Similarity} \;+\;
0.30 \cdot \text{Average Rating (scaled)} \;+\;
0.20 \cdot \text{Sentiment (scaled)} \;+\;
0.10 \cdot \text{Review Count (scaled)}
$$

We then ranked the beers by this score. Based on this, our top 3 recommendations are:
1. Triple Shot
2. KBS
3. Parabajava

Other contenders that were listed in the Top-20 ranked table include: Speedway Stout - Bourbon Barrel Aged, Kaggen!, Stormaktsporter, Breakfast Stout, and CBS.

We've added in a Top-20 ranked table, with the Top-3 beers flagged, reported cosine similarity, average rating, sentiment, review count, and the composite score, to make the recommendation process transparent.




Task C. How would your recommendations change if you use word vectors (e.g., the spaCy package with
medium sized pretrained word vectors) instead of the plain vanilla bag-of-words cosine similarity?
In your analysis, which method gives you better recommendations in this context? Explain your
response.
This article may be useful:
Word Embeddings Versus Bag-of-Words: The Curious Case of Recommender Systems | by Josh Barua |
The Startup | Medium
Note that the article doesn’t claim that bag-of-words will always be better than word embeddings for
recommender systems. It lays out conditions under which it is likely to be the case. That is, depending
on the attributes you use, you may or may not see the same effect.


In [ ]:
"""
Task C: Word Vectors vs Bag-of-Words Comparison
REQUIRES: Task B to be completed first

Question: How would your recommendations change if you use word vectors
instead of the plain vanilla bag-of-words cosine similarity?
"""

import pandas as pd
import numpy as np
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

print("\n" + "="*80)
print("Task C: Word Vectors vs Bag-of-Words Comparison")
print("="*80)

# ================================
# STEP 1: VALIDATE TASK B COMPLETION
# ================================

try:
    # Check if Task B variables exist
    test_vars = [selected_attributes, beer_profiles, recommendations]
    print(f"Task B baseline found:")
    print(f"  Customer attributes: {selected_attributes}")
    print(f"  Beer profiles: {len(beer_profiles)} beers")
    print(f"  TF-IDF recommendations: {len(recommendations)} beers")
except NameError as e:
    print(f"Error: Task B must be completed first!")
    print(f"Missing variable: {e}")
    print(f"Please run Task B script before running Task C")
    exit(1)

# ================================
# STEP 2: IMPLEMENT WORD VECTOR METHOD
# ================================

print(f"\nImplementing word vector approach...")

def load_spacy_model():
    """Load spaCy model with error handling"""
    try:
        print("Loading spaCy medium model (en_core_web_md)...")
        nlp = spacy.load("en_core_web_md")
        print(f"  Vector dimensions: {nlp.vocab.vectors_length}")
        print(f"  Vocabulary size: {len(nlp.vocab.vectors)}")
        return nlp
    except IOError:
        print("Error: spaCy medium model not found!")
        print("Install with: python -m spacy download en_core_web_md")
        exit(1)

nlp = load_spacy_model()

def get_text_embedding(text, nlp_model):
    """Create document embedding by averaging word vectors"""
    if not isinstance(text, str) or len(text.strip()) == 0:
        return np.zeros(nlp_model.vocab.vectors_length)

    doc = nlp_model(text)

    # Get vectors for meaningful tokens
    vectors = []
    for token in doc:
        if (token.has_vector and
            not token.is_stop and
            not token.is_punct and
            not token.is_space and
            len(token.text) > 2):
            vectors.append(token.vector)

    if vectors:
        # Average the word vectors and normalize
        embedding = np.mean(vectors, axis=0)
        norm = np.linalg.norm(embedding)
        if norm > 0:
            embedding = embedding / norm
        return embedding
    else:
        return np.zeros(nlp_model.vocab.vectors_length)

# Create customer embedding
print("Creating customer preference embedding...")
customer_profile = ' '.join(selected_attributes * 3)
customer_embedding = get_text_embedding(customer_profile, nlp)

# Create beer embeddings
print("Creating beer profile embeddings...")
beer_embeddings = []
beer_names = []

for i, (beer_name, review_text) in enumerate(beer_profiles['clean_review'].items()):
    if i % 50 == 0:
        print(f"  Processing beer {i+1}/{len(beer_profiles)}...")

    embedding = get_text_embedding(review_text, nlp)
    beer_embeddings.append(embedding)
    beer_names.append(beer_name)

beer_embeddings = np.array(beer_embeddings)

# Calculate cosine similarities with word vectors
print("Computing word vector similarities...")
customer_embedding_reshaped = customer_embedding.reshape(1, -1)
wv_similarities = cosine_similarity(customer_embedding_reshaped, beer_embeddings).flatten()

# Create word vector recommendations
wv_recommendations = pd.DataFrame({
    'beer_name': beer_names,
    'wv_similarity': wv_similarities,
    'avg_rating': beer_profiles['user_rating'],
    'avg_sentiment': beer_profiles['sentiment'],
    'review_count': beer_profiles['review_count']
})

# Calculate composite score (same weights as Task B)
wv_recommendations['wv_composite_score'] = (
    0.4 * wv_recommendations['wv_similarity'] +
    0.3 * (wv_recommendations['avg_rating'] / 5.0) +
    0.2 * ((wv_recommendations['avg_sentiment'] + 1) / 2) +
    0.1 * np.log(wv_recommendations['review_count'] + 1) / 10
)

wv_recommendations = wv_recommendations.sort_values('wv_composite_score', ascending=False)

print(f"Word vector recommendations generated")

# ================================
# STEP 3: COMPARE HOW RECOMMENDATIONS CHANGED
# ================================

print(f"\n{'='*80}")
print("ANALYSIS: HOW RECOMMENDATIONS CHANGED")
print(f"{'='*80}")

print(f"Customer request: 'I want a beer that is {', '.join(selected_attributes)}'")

# Get top 10 from both methods
tfidf_top10 = recommendations.head(10)
wv_top10 = wv_recommendations.head(10)

# Find overlaps
tfidf_beers = set(tfidf_top10['beer_name'])
wv_beers = set(wv_top10['beer_name'])
common_beers = tfidf_beers.intersection(wv_beers)
tfidf_only = tfidf_beers - wv_beers
wv_only = wv_beers - tfidf_beers

print(f"\nOverlap Analysis:")
print(f"  Common beers in both top 10s: {len(common_beers)} ({len(common_beers)/10:.0%})")
print(f"  Only in TF-IDF top 10: {len(tfidf_only)}")
print(f"  Only in Word Vectors top 10: {len(wv_only)}")

if len(common_beers) > 0:
    print(f"  Common beers: {', '.join(list(common_beers)[:3])}...")

# Side-by-side comparison
print(f"\n{'='*100}")
print("SIDE-BY-SIDE COMPARISON: TOP 10 RECOMMENDATIONS")
print(f"{'='*100}")

print(f"\nTF-IDF BAG-OF-WORDS (Task B Baseline):")
print(f"{'Rank':<4} {'Beer Name':<45} {'Similarity':<10} {'Score':<8}")
print("-" * 75)
for i, (_, row) in enumerate(tfidf_top10.iterrows(), 1):
    marker = "✓" if row['beer_name'] in common_beers else " "
    print(f"{i:<4} {row['beer_name'][:44]:<45} {row['cosine_similarity']:.3f}      {row['composite_score']:.3f} {marker}")

print(f"\nWORD VECTORS (SpaCy Alternative):")
print(f"{'Rank':<4} {'Beer Name':<45} {'Similarity':<10} {'Score':<8}")
print("-" * 75)
for i, (_, row) in enumerate(wv_top10.iterrows(), 1):
    marker = "✓" if row['beer_name'] in common_beers else " "
    print(f"{i:<4} {row['beer_name'][:44]:<45} {row['wv_similarity']:.3f}      {row['wv_composite_score']:.3f} {marker}")

print(f"\n✓ = Appears in both top 10 lists")

# Statistical comparison
print(f"\n{'='*80}")
print("STATISTICAL DIFFERENCES")
print(f"{'='*80}")

tfidf_sim_mean = recommendations['cosine_similarity'].mean()
wv_sim_mean = wv_recommendations['wv_similarity'].mean()
tfidf_sim_std = recommendations['cosine_similarity'].std()
wv_sim_std = wv_recommendations['wv_similarity'].std()

print(f"Similarity Score Distributions:")
print(f"  TF-IDF Bag-of-Words:  Mean={tfidf_sim_mean:.3f}, Std={tfidf_sim_std:.3f}")
print(f"  Word Vectors:         Mean={wv_sim_mean:.3f}, Std={wv_sim_std:.3f}")
print(f"  Difference in means:  {abs(wv_sim_mean - tfidf_sim_mean):.3f}")

# Top recommendation comparison
tfidf_top1 = recommendations.iloc[0]
wv_top1 = wv_recommendations.iloc[0]

print(f"\nTop Recommendation Comparison:")
print(f"  TF-IDF #1: {tfidf_top1['beer_name']} (similarity: {tfidf_top1['cosine_similarity']:.3f})")
print(f"  Word Vector #1: {wv_top1['beer_name']} (similarity: {wv_top1['wv_similarity']:.3f})")
print(f"  Same top beer: {'Yes' if tfidf_top1['beer_name'] == wv_top1['beer_name'] else 'No'}")

# ================================
# STEP 4: RESULTS SUMMARY
# ================================

print(f"\n{'='*80}")
print("RESULTS SUMMARY")
print(f"{'='*80}")

print(f"Context Analysis for Beer Recommendations:")

# Check attribute coverage in the data
attribute_coverage = {}
for attr in selected_attributes:
    count = sum(1 for text in beer_profiles['clean_review'] if attr in text.lower())
    coverage = count / len(beer_profiles)
    attribute_coverage[attr] = coverage
    print(f"  '{attr}': Found in {count}/{len(beer_profiles)} beers ({coverage:.1%})")

avg_coverage = sum(attribute_coverage.values()) / len(attribute_coverage)
print(f"  Average attribute coverage: {avg_coverage:.1%}")

print(f"\nMethod Agreement Analysis:")
agreement_rate = len(common_beers) / 10
if agreement_rate >= 0.7:
    agreement_level = "High"
elif agreement_rate >= 0.4:
    agreement_level = "Moderate"
else:
    agreement_level = "Low"

print(f"  Agreement rate: {agreement_rate:.0%} ({agreement_level})")

print(f"\n{'='*80}")
print("SUMMARY: HOW RECOMMENDATIONS CHANGED")
print(f"{'='*80}")

print(f"When switching from TF-IDF bag-of-words to word vectors:")
print(f"  • {len(common_beers)} beers remained in top 10 ({agreement_rate:.0%} consistency)")
print(f"  • {len(tfidf_only)} beers dropped out of top 10")
print(f"  • {len(wv_only)} new beers entered top 10")
print(f"  • Top recommendation {'stayed the same' if tfidf_top1['beer_name'] == wv_top1['beer_name'] else 'changed'}")

print(f"\nTask C Complete: Word vectors vs bag-of-words comparison finished!")
print(f"TF-IDF top-3: {', '.join(tfidf_top10.head(3)['beer_name'].tolist())}")
print(f"Word Vector top-3: {', '.join(wv_top10.head(3)['beer_name'].tolist())}")


Task C: Word Vectors vs Bag-of-Words Comparison
Task B baseline found:
  Customer attributes: ['chocolate', 'dark', 'coffee']
  Beer profiles: 249 beers
  TF-IDF recommendations: 243 beers

Implementing word vector approach...
Loading spaCy medium model (en_core_web_md)...
  Vector dimensions: 300
  Vocabulary size: 20000
Creating customer preference embedding...
Creating beer profile embeddings...
  Processing beer 1/249...
  Processing beer 51/249...
  Processing beer 101/249...
  Processing beer 151/249...
  Processing beer 201/249...
Computing word vector similarities...
Word vector recommendations generated

ANALYSIS: HOW RECOMMENDATIONS CHANGED
Customer request: 'I want a beer that is chocolate, dark, coffee'

Overlap Analysis:
  Common beers in both top 10s: 5 (50%)
  Only in TF-IDF top 10: 5
  Only in Word Vectors top 10: 5
  Common beers: Triple Shot, CBS (Canadian Breakfast Stout), Coffee Cinnamon B-Bomb...

SIDE-BY-SIDE COMPARISON: TOP 10 RECOMMENDATIONS

TF-IDF BAG-OF-WORD

In **Task C**, we used the spaCy package's pretrained word embeddings instead of the plain vanilla bag-of-words cosine similarity. Unlike the BoW model (which only looks at the words that appear), the spaCy package places words in a vector space where similar words are close together (“bourbon/vanilla near chocolate/dark”).

We then created a document vector for each beer by averaging the vectors of all meaningful words in its reviews, and a query vector for the attributes (chocolate, dark, coffee) in the same way. We then computed cosine similarity and combined it with rating, sentiment, and review count using the same composite weighting as we did in Task B. This let us measure how closely the language in each beer’s reviews matched the customer’s desired attributes.

The results showed some overlap between the two methods' Top-10 lists. Results:
- Top-3 (TF-IDF BoW): TripleShot, KBS, Parabajava
- Top-3 (Word Vectors): KBS, Triple Shot, V.S.O.J
- Top-10 overlap: 5/10 beers in common → 50% overlap
- #1 recommendation: Changed (relative to Task B); from TripleShot to KBS

With the chocolate/dark/coffee, the Bag-of-Words model generates specific keyword matches whereas the word vectors generates/ focuses on reviews that use related or similar descriptors but aren't necessarily 1:1 (i.e barrel/ vanilla). This, in turn, lifted beers such as V.S.O.J into the Top-3 and moved KBS to the number 1 spot. Here, found that both methods are ultimately useful- bag of words makes sure that the top picks are aligned to the stated/top attirbutes, while the word embeddings contextualize the output and add semantic coverage. However, a combination or a hybrid model is likely best suited.

**Task D**: Create custom word embeddings from your product review data instead of using the default SpaCy word embeddings. Do your top-3 recommendations change when using your own embeddings?
You can use either SpaCy or Gensim in creating your custom embeddings.

In [ ]:
"""
Task D: Create custom word embeddings from product review data
REQUIRES: Tasks B and C to be completed first

Question: Do your top-3 recommendations change when using your own embeddings?
"""

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import re
import warnings
warnings.filterwarnings('ignore')

print("\n" + "="*80)
print("Task D: Custom Word Embeddings from Beer Review Data")
print("="*80)

# ================================
# STEP 1: VALIDATE PREVIOUS TASKS
# ================================

try:
    # Check if previous task variables exist
    test_vars = [selected_attributes, beer_profiles, recommendations, wv_recommendations]
    print("Previous tasks found:")
    print(f"  Task B (TF-IDF): {len(recommendations)} recommendations")
    print(f"  Task C (spaCy): {len(wv_recommendations)} recommendations")
    print(f"  Customer attributes: {selected_attributes}")
except NameError as e:
    print(f"Error: Previous tasks must be completed first!")
    print(f"Missing variable: {e}")
    print("Please run Tasks B and C before running Task D")
    exit(1)

# ================================
# STEP 2: PREPARE TEXT DATA FOR TRAINING
# ================================

print(f"\nPreparing text data for custom embedding training...")

def prepare_sentences_for_training(beer_profiles):
    """
    Prepare beer review text for Word2Vec training
    Tokenizes and cleans text into sentences of words
    """

    def clean_and_tokenize(text):
        """Clean text and return list of words"""
        if not isinstance(text, str):
            return []

        # Convert to lowercase and extract words
        text = text.lower()
        # Keep only alphabetic words of reasonable length
        words = re.findall(r'\b[a-z]{2,}\b', text)

        # Filter out very common stop words (keep beer-relevant ones)
        basic_stops = {'the', 'and', 'but', 'for', 'are', 'was', 'were', 'been',
                      'have', 'has', 'had', 'this', 'that', 'with', 'from', 'they',
                      'them', 'than', 'very', 'more', 'most', 'some', 'what', 'when'}

        words = [word for word in words if word not in basic_stops and len(word) >= 3]

        return words

    # Collect all sentences from beer reviews
    sentences = []
    total_words = 0

    print("Processing beer review text...")
    for i, (beer_name, review_text) in enumerate(beer_profiles['clean_review'].items()):
        if i % 100 == 0:
            print(f"  Processing beer {i+1}/{len(beer_profiles)}...")

        # Split long review into smaller chunks (Word2Vec works better with shorter sentences)
        if isinstance(review_text, str) and len(review_text) > 100:
            # Split by sentence-ending punctuation
            chunks = re.split(r'[.!?]+', review_text)

            for chunk in chunks:
                words = clean_and_tokenize(chunk)
                if len(words) >= 5:  # Minimum sentence length
                    sentences.append(words)
                    total_words += len(words)

    print(f"Training data prepared:")
    print(f"  Total sentences: {len(sentences)}")
    print(f"  Total words: {total_words}")
    print(f"  Average words per sentence: {total_words/len(sentences):.1f}")

    return sentences

# Prepare training data
training_sentences = prepare_sentences_for_training(beer_profiles)

if len(training_sentences) < 100:
    print("Warning: Very small training corpus. Custom embeddings may be poor quality.")
    print("Consider using pre-trained embeddings for better results.")

# ================================
# STEP 3: TRAIN CUSTOM WORD2VEC EMBEDDINGS
# ================================

print(f"\nTraining custom Word2Vec embeddings...")

def train_custom_embeddings(sentences, vector_size=100, window=5, min_count=5, epochs=20):
    """
    Train custom Word2Vec embeddings on beer review data

    Parameters:
    - vector_size: Dimensionality of word vectors
    - window: Context window size
    - min_count: Ignore words with frequency < min_count
    - epochs: Number of training iterations
    """

    print(f"Word2Vec parameters:")
    print(f"  Vector size: {vector_size}")
    print(f"  Context window: {window}")
    print(f"  Min word count: {min_count}")
    print(f"  Training epochs: {epochs}")

    # Train Word2Vec model
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        workers=4,
        epochs=epochs,
        sg=1,  # Skip-gram model (better for small datasets)
        seed=42  # For reproducibility
    )

    print(f"Custom embeddings trained:")
    print(f"  Vocabulary size: {len(model.wv.key_to_index)}")
    print(f"  Vector dimensions: {model.wv.vector_size}")

    return model

# Train the custom embedding model
custom_w2v_model = train_custom_embeddings(training_sentences)

# Check if customer attributes are in vocabulary
print(f"\nCustomer attribute coverage in custom embeddings:")
missing_attributes = []
for attr in selected_attributes:
    if attr in custom_w2v_model.wv.key_to_index:
        print(f"  ✓ '{attr}': Found in vocabulary")
    else:
        print(f"  ✗ '{attr}': Missing from vocabulary")
        missing_attributes.append(attr)

if missing_attributes:
    print(f"Warning: {len(missing_attributes)} customer attributes missing from vocabulary")
    print("This may affect recommendation quality")

# ================================
# STEP 4: CREATE DOCUMENT EMBEDDINGS
# ================================

print(f"\nCreating document embeddings using custom Word2Vec...")

def get_custom_text_embedding(text, w2v_model):
    """
    Create document embedding using custom Word2Vec model
    Average word vectors for words in vocabulary
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return np.zeros(w2v_model.wv.vector_size)

    # Tokenize text
    words = re.findall(r'\b[a-z]{3,}\b', text.lower())

    # Get vectors for words in vocabulary
    vectors = []
    for word in words:
        if word in w2v_model.wv.key_to_index:
            vectors.append(w2v_model.wv[word])

    if vectors:
        # Average word vectors and normalize
        embedding = np.mean(vectors, axis=0)
        norm = np.linalg.norm(embedding)
        if norm > 0:
            embedding = embedding / norm
        return embedding
    else:
        return np.zeros(w2v_model.wv.vector_size)

# Create customer embedding with custom model
print("Creating customer preference embedding...")
customer_profile = ' '.join(selected_attributes * 3)
custom_customer_embedding = get_custom_text_embedding(customer_profile, custom_w2v_model)

# Create beer embeddings with custom model
print("Creating beer profile embeddings...")
custom_beer_embeddings = []
custom_beer_names = []

for i, (beer_name, review_text) in enumerate(beer_profiles['clean_review'].items()):
    if i % 50 == 0:
        print(f"  Processing beer {i+1}/{len(beer_profiles)}...")

    embedding = get_custom_text_embedding(review_text, custom_w2v_model)
    custom_beer_embeddings.append(embedding)
    custom_beer_names.append(beer_name)

custom_beer_embeddings = np.array(custom_beer_embeddings)

# Calculate similarities
print("Computing similarities with custom embeddings...")
custom_customer_reshaped = custom_customer_embedding.reshape(1, -1)
custom_similarities = cosine_similarity(custom_customer_reshaped, custom_beer_embeddings).flatten()

# ================================
# STEP 5: GENERATE CUSTOM EMBEDDING RECOMMENDATIONS
# ================================

print(f"\nGenerating recommendations with custom embeddings...")

# Create custom embedding recommendations
custom_recommendations = pd.DataFrame({
    'beer_name': custom_beer_names,
    'custom_similarity': custom_similarities,
    'avg_rating': beer_profiles['user_rating'],
    'avg_sentiment': beer_profiles['sentiment'],
    'review_count': beer_profiles['review_count']
})

# Calculate composite score (same weights as other methods)
custom_recommendations['custom_composite_score'] = (
    0.4 * custom_recommendations['custom_similarity'] +
    0.3 * (custom_recommendations['avg_rating'] / 5.0) +
    0.2 * ((custom_recommendations['avg_sentiment'] + 1) / 2) +
    0.1 * np.log(custom_recommendations['review_count'] + 1) / 10
)

# Sort by composite score
custom_recommendations = custom_recommendations.sort_values('custom_composite_score', ascending=False)

print(f"Custom embedding recommendations generated")

# ================================
# STEP 6: COMPARE TOP-3 RECOMMENDATIONS
# ================================

print(f"\n{'='*80}")
print("COMPARISON: DO TOP-3 RECOMMENDATIONS CHANGE?")
print(f"{'='*80}")

# Get top 3 from all methods
tfidf_top3 = recommendations.head(3)
spacy_top3 = wv_recommendations.head(3)
custom_top3 = custom_recommendations.head(3)

print(f"Customer request: 'I want a beer that is {', '.join(selected_attributes)}'")

print(f"\nTOP-3 COMPARISON ACROSS ALL METHODS:")
print(f"{'Method':<20} {'#1 Recommendation':<30} {'#2 Recommendation':<30} {'#3 Recommendation':<30}")
print("-" * 110)

methods_data = [
    ("TF-IDF (Task B)", tfidf_top3),
    ("spaCy Vectors (Task C)", spacy_top3),
    ("Custom Embeddings", custom_top3)
]

for method_name, top3_df in methods_data:
    rec1 = top3_df.iloc[0]['beer_name'][:29]
    rec2 = top3_df.iloc[1]['beer_name'][:29]
    rec3 = top3_df.iloc[2]['beer_name'][:29]
    print(f"{method_name:<20} {rec1:<30} {rec2:<30} {rec3:<30}")

# Analyze changes
print(f"\n{'='*80}")
print("CHANGE ANALYSIS")
print(f"{'='*80}")

# Compare custom vs spaCy (most relevant comparison)
spacy_top3_names = set(spacy_top3['beer_name'])
custom_top3_names = set(custom_top3['beer_name'])
tfidf_top3_names = set(tfidf_top3['beer_name'])

spacy_custom_overlap = spacy_top3_names.intersection(custom_top3_names)
tfidf_custom_overlap = tfidf_top3_names.intersection(custom_top3_names)

print(f"Custom Embeddings vs spaCy Pre-trained:")
print(f"  Common beers in top-3: {len(spacy_custom_overlap)}/3")
print(f"  Overlap percentage: {len(spacy_custom_overlap)/3:.0%}")

print(f"\nCustom Embeddings vs TF-IDF:")
print(f"  Common beers in top-3: {len(tfidf_custom_overlap)}/3")
print(f"  Overlap percentage: {len(tfidf_custom_overlap)/3:.0%}")

# Show detailed top-3 from custom embeddings
print(f"\n{'='*80}")
print("DETAILED TOP-3 WITH CUSTOM EMBEDDINGS")
print(f"{'='*80}")

for i, (_, beer) in enumerate(custom_top3.iterrows(), 1):
    status_indicators = []
    if beer['beer_name'] in spacy_top3_names:
        status_indicators.append("spaCy")
    if beer['beer_name'] in tfidf_top3_names:
        status_indicators.append("TF-IDF")

    status = " (also in: " + ", ".join(status_indicators) + ")" if status_indicators else " (unique to custom)"

    print(f"\n{i}. {beer['beer_name']}{status}")
    print(f"   Custom Similarity: {beer['custom_similarity']:.3f}")
    print(f"   Average Rating: {beer['avg_rating']:.2f}/5.0")
    print(f"   Sentiment Score: {beer['avg_sentiment']:.3f}")
    print(f"   Final Score: {beer['custom_composite_score']:.3f}")

# ================================
# STEP 7: ANALYZE EMBEDDING QUALITY
# ================================

print(f"\n{'='*80}")
print("CUSTOM EMBEDDING QUALITY ANALYSIS")
print(f"{'='*80}")

# Check similarity statistics
custom_sim_mean = custom_recommendations['custom_similarity'].mean()
custom_sim_std = custom_recommendations['custom_similarity'].std()
spacy_sim_mean = wv_recommendations['wv_similarity'].mean()
spacy_sim_std = wv_recommendations['wv_similarity'].std()

print(f"Similarity Score Statistics:")
print(f"  Custom Embeddings: Mean={custom_sim_mean:.3f}, Std={custom_sim_std:.3f}")
print(f"  spaCy Pre-trained: Mean={spacy_sim_mean:.3f}, Std={spacy_sim_std:.3f}")
print(f"  Difference: {abs(custom_sim_mean - spacy_sim_mean):.3f}")

# Test custom embedding quality with beer terms
print(f"\nCustom Embedding Quality Check:")

if len(custom_w2v_model.wv.key_to_index) >= 50:
    # Find similar words to customer attributes (if they exist)
    for attr in selected_attributes:
        if attr in custom_w2v_model.wv.key_to_index:
            try:
                similar_words = custom_w2v_model.wv.most_similar(attr, topn=3)
                similar_list = [word for word, score in similar_words]
                print(f"  '{attr}' similar to: {', '.join(similar_list)}")
            except:
                print(f"  '{attr}': Cannot find similar words")
        else:
            print(f"  '{attr}': Not in custom vocabulary")

    # Check vocabulary coverage
    vocab_size = len(custom_w2v_model.wv.key_to_index)
    if vocab_size < 500:
        quality_assessment = "Limited"
        print(f"\nQuality Assessment: {quality_assessment}")
        print(f"  Small vocabulary ({vocab_size} words) may limit recommendation quality")
    elif vocab_size < 2000:
        quality_assessment = "Moderate"
        print(f"\nQuality Assessment: {quality_assessment}")
        print(f"  Reasonable vocabulary ({vocab_size} words) for domain-specific embeddings")
    else:
        quality_assessment = "Good"
        print(f"\nQuality Assessment: {quality_assessment}")
        print(f"  Large vocabulary ({vocab_size} words) should provide good coverage")
else:
    quality_assessment = "Poor"
    print(f"\nQuality Assessment: {quality_assessment}")
    print("Very small vocabulary - embeddings may not be reliable")

# ================================
# STEP 8: FINAL RESULTS
# ================================

print(f"\n{'='*80}")
print("FINAL RESULTS")
print(f"{'='*80}")

print(f"Answer: Do top-3 recommendations change when using custom embeddings?")

if len(spacy_custom_overlap) == 3:
    change_assessment = "No - all 3 recommendations remain the same"
elif len(spacy_custom_overlap) >= 2:
    change_assessment = f"Minor changes - {3-len(spacy_custom_overlap)} out of 3 recommendations changed"
else:
    change_assessment = f"Major changes - {3-len(spacy_custom_overlap)} out of 3 recommendations changed"

print(f"  {change_assessment}")

print(f"\nTask D Complete: Custom embeddings analysis finished!")
print(f"Custom top-3: {', '.join(custom_top3['beer_name'].tolist())}")
print(f"spaCy top-3: {', '.join(spacy_top3['beer_name'].tolist())}")
print(f"TF-IDF top-3: {', '.join(tfidf_top3['beer_name'].tolist())}")


Task D: Custom Word Embeddings from Beer Review Data
Previous tasks found:
  Task B (TF-IDF): 243 recommendations
  Task C (spaCy): 249 recommendations
  Customer attributes: ['chocolate', 'dark', 'coffee']

Preparing text data for custom embedding training...
Processing beer review text...
  Processing beer 1/249...
  Processing beer 101/249...
  Processing beer 201/249...
Training data prepared:
  Total sentences: 249
  Total words: 153930
  Average words per sentence: 618.2

Training custom Word2Vec embeddings...
Word2Vec parameters:
  Vector size: 100
  Context window: 5
  Min word count: 5
  Training epochs: 20
Custom embeddings trained:
  Vocabulary size: 2689
  Vector dimensions: 100

Customer attribute coverage in custom embeddings:
  ✓ 'chocolate': Found in vocabulary
  ✓ 'dark': Found in vocabulary
  ✓ 'coffee': Found in vocabulary

Creating document embeddings using custom Word2Vec...
Creating customer preference embedding...
Creating beer profile embeddings...
  Processing

In **Task D**, we created custom word embeddings from our product review data instead of using the default SpaCY word embeddings. Since these were from our dataset, they capture beer-specific semantics and language patterns.

Similar to Task B and C, we created a vector for each beer (by averaging its review word vectors) and a query vector for the customer’s attributes (chocolate, dark, coffee). We then computed cosine similarity and combined it with rating, sentiment, and review count using the same composite formula from Task B.

The results:

Top-3 (TF-IDF BoW): Triple Shot, KBS, Parabajava

Top-3 (spaCy vectors): KBS, Triple Shot, V.S.O.J.

Top-3 (Custom Word2Vec): V.S.O.J., CBS (Canadian Breakfast Stout), Abraxas – Barrel-Aged

Overlap with spaCy: 1/3

Overlap with TF-IDF: 0/3

The #1 recommendation: This changed again, this time to V.S.O.J.

Overall, the custom embeddings surfaced different beers because they focused on how words are actually used in beer reviews and focused on the context. Compared to BoW and spaCy, this method pushed V.S.O.J., CBS, and Abraxas – Barrel-Aged into the top results, showing that training on domain data can meaningfully change recommendations.

Task E. How would your recommendations differ if you ignored the similarity and feature sentiment
scores and simply chose the 3 highest rated products from your entire dataset? Would these products
meet the requirements of the user looking for recommendations? Why or why not? Justify your answer
with analysis.

In [ ]:
"""
Task E: Compare personalized recommendations vs highest-rated products

Question: How would your recommendations differ if you ignored similarity and sentiment
and simply chose the 3 highest rated products? Would these meet user requirements?
"""

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("\n" + "="*60)
print("Task E: Highest Rated vs Personalized Recommendations")
print("="*60)

# ================================
# STEP 1: VALIDATE PREVIOUS TASKS
# ================================

try:
    test_vars = [selected_attributes, beer_profiles, recommendations]
    print("Previous tasks found:")
    print(f"  Customer attributes: {selected_attributes}")
    print(f"  Personalized recommendations: {len(recommendations)} beers")
except NameError as e:
    print(f"Error: Previous tasks must be completed first!")
    exit(1)

# ================================
# STEP 2: CREATE HIGHEST-RATED RECOMMENDATIONS
# ================================

print(f"\nCreating highest-rated recommendations...")

# Simple approach: sort by rating, take top 3
highest_rated = beer_profiles.sort_values('user_rating', ascending=False)
highest_rated_top3 = highest_rated.head(3)

print(f"Highest-rated approach: Simply sort by rating and take top 3")

# ================================
# STEP 3: COMPARISON
# ================================

print(f"\n{'='*80}")
print("COMPARISON: PERSONALIZED vs HIGHEST-RATED")
print(f"{'='*80}")

print(f"Customer request: 'I want a beer that is {', '.join(selected_attributes)}'")

# Get personalized top 3
personalized_top3 = recommendations.head(3)

# Check overlap
personalized_names = set(personalized_top3['beer_name'])
highest_rated_names = set(highest_rated_top3.index)
overlap = personalized_names.intersection(highest_rated_names)

print(f"\nOverlap: {len(overlap)}/3 beers appear in both lists ({len(overlap)/3:.0%})")
if len(overlap) > 0:
    print(f"Common beers: {', '.join(overlap)}")

# Side-by-side comparison
print(f"\nPERSONALIZED TOP 3 (considers customer preferences):")
print(f"{'Rank':<4} {'Beer Name':<40} {'Rating':<8} {'Similarity':<10}")
print("-" * 70)
for i, (_, row) in enumerate(personalized_top3.iterrows(), 1):
    marker = "✓" if row['beer_name'] in overlap else " "
    print(f"{i:<4} {row['beer_name'][:39]:<40} {row['avg_rating']:.2f}     {row['cosine_similarity']:.3f} {marker}")

print(f"\nHIGHEST-RATED TOP 3 (ignores customer preferences):")
print(f"{'Rank':<4} {'Beer Name':<40} {'Rating':<8} {'Reviews':<8}")
print("-" * 70)
for i, (beer_name, row) in enumerate(highest_rated_top3.iterrows(), 1):
    marker = "✓" if beer_name in overlap else " "
    print(f"{i:<4} {beer_name[:39]:<40} {row['user_rating']:.2f}     {int(row['review_count']):<8} {marker}")

print(f"\n✓ = Appears in both lists")

# ================================
# STEP 4: ATTRIBUTE MATCHING ANALYSIS
# ================================

print(f"\n{'='*60}")
print("ATTRIBUTE MATCHING ANALYSIS")
print(f"{'='*60}")

def count_attribute_mentions(beer_names, beer_profiles, attributes):
    """Count how often customer attributes appear in beer reviews"""
    results = []

    for beer_name in beer_names:
        if beer_name in beer_profiles.index:
            review_text = beer_profiles.loc[beer_name, 'clean_review']
            total_matches = 0

            for attr in attributes:
                count = review_text.lower().count(attr.lower())
                total_matches += count

            results.append({
                'beer_name': beer_name,
                'total_matches': total_matches
            })

    return results

# Analyze attribute matching
personalized_matches = count_attribute_mentions(
    personalized_top3['beer_name'].tolist(),
    beer_profiles,
    selected_attributes
)

highest_rated_matches = count_attribute_mentions(
    highest_rated_top3.index.tolist(),
    beer_profiles,
    selected_attributes
)

print(f"Customer attribute mentions ('{', '.join(selected_attributes)}'):")

print(f"\nPersonalized recommendations:")
for result in personalized_matches:
    print(f"  {result['beer_name'][:35]}: {result['total_matches']} mentions")

print(f"\nHighest-rated recommendations:")
for result in highest_rated_matches:
    print(f"  {result['beer_name'][:35]}: {result['total_matches']} mentions")

# Calculate averages
personalized_avg = np.mean([r['total_matches'] for r in personalized_matches])
highest_rated_avg = np.mean([r['total_matches'] for r in highest_rated_matches])

print(f"\nAverage attribute mentions:")
print(f"  Personalized: {personalized_avg:.1f}")
print(f"  Highest-rated: {highest_rated_avg:.1f}")
print(f"  Difference: {personalized_avg - highest_rated_avg:+.1f}")

# ================================
# STEP 5: RATING COMPARISON
# ================================

print(f"\n{'='*60}")
print("RATING COMPARISON")
print(f"{'='*60}")

personalized_ratings = personalized_top3['avg_rating']
highest_rated_ratings = highest_rated_top3['user_rating']

print(f"Average ratings:")
print(f"  Personalized: {personalized_ratings.mean():.2f}/5.0")
print(f"  Highest-rated: {highest_rated_ratings.mean():.2f}/5.0")
print(f"  Rating sacrifice: {highest_rated_ratings.mean() - personalized_ratings.mean():+.2f}")

# ================================
# STEP 6: SIMPLE CONCLUSION
# ================================

print(f"\n{'='*60}")
print("ANALYSIS SUMMARY")
print(f"{'='*60}")

print(f"Question: Would highest-rated products meet user requirements?")

# Simple decision logic
if highest_rated_avg >= personalized_avg * 0.8:
    relevance_assessment = "adequate"
elif highest_rated_avg >= personalized_avg * 0.5:
    relevance_assessment = "poor"
else:
    relevance_assessment = "very poor"

rating_sacrifice = highest_rated_ratings.mean() - personalized_ratings.mean()

print(f"\nFindings:")
print(f"  Overlap: {len(overlap)}/3 beers are the same")
print(f"  Attribute relevance: {relevance_assessment}")
print(f"  Rating difference: {rating_sacrifice:+.2f} points")

print(f"\nConclusion: Highest-rated approach would provide {relevance_assessment} relevance to user requirements")

print(f"\nTask E Complete!")
print(f"Personalized top beer: {personalized_top3.iloc[0]['beer_name']}")
print(f"Highest-rated top beer: {highest_rated_top3.index[0]}")


Task E: Highest Rated vs Personalized Recommendations
Previous tasks found:
  Customer attributes: ['chocolate', 'dark', 'coffee']
  Personalized recommendations: 243 beers

Creating highest-rated recommendations...
Highest-rated approach: Simply sort by rating and take top 3

COMPARISON: PERSONALIZED vs HIGHEST-RATED
Customer request: 'I want a beer that is chocolate, dark, coffee'

Overlap: 0/3 beers appear in both lists (0%)

PERSONALIZED TOP 3 (considers customer preferences):
Rank Beer Name                                Rating   Similarity
----------------------------------------------------------------------
1    Triple Shot                              4.50     0.621  
2    KBS                                      4.65     0.489  
3    Parabajava                               4.55     0.417  

HIGHEST-RATED TOP 3 (ignores customer preferences):
Rank Beer Name                                Rating   Reviews 
----------------------------------------------------------------------

In Task E, we compared our personalized recommendation method against a baseline that ignores similarity and sentiment, and instead simply selects the three highest-rated beers in the dataset.

Our personalized model (Task B) identified Triple Shot, KBS, and Parabajava as the top recommendations for a customer wanting a beer that is chocolate, dark, and coffee. In contrast, the highest-rated baseline surfaced 10 Year Barleywine, M.J.K., and O.W.K., all of which had excellent ratings (average 4.94/5.0) but almost no mentions of the customer’s desired attributes. Additionally, there were no beers in common between these models.

Overall, the results show that while the highest-rated products are objectively strong beers, they provide very poor relevance to the customer’s specific preferences. This emphasizes the value of incorporating similarity and sentiment signals: recommendations should balance quality with alignment to user requirements, and not just rely on ratings alone.

Task F. Choose any 10 beers in your data. Now choose any one of them, and find the most similar beer
(among the remaining 9). Explain your method and logic.

In [ ]:
"""
Task F: Simple Beer-to-Beer Content Similarity (Fixed)
"""

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

print("Task F: Direct Beer Content Similarity (Fixed)")

# Load and validate data
df = pd.read_csv('beeradvocate_reviews.csv')

# Clean and validate beer names
def is_valid_beer_name(name):
    if pd.isna(name) or not isinstance(name, str):
        return False

    # Filter out timestamps and metadata
    invalid_patterns = [
        r'\d+\s*(minutes?|hours?|days?|weeks?)\s*ago',  # timestamps
        r'character count:',  # system messages
        r'thanks for the review',
        r'report',
        r'^\s*$',  # empty strings
        r'^[0-9\.\s]+$'  # just numbers
    ]

    name_lower = name.lower().strip()
    for pattern in invalid_patterns:
        if re.search(pattern, name_lower):
            return False

    return len(name.strip()) > 3  # Minimum reasonable length

# Filter for valid beers
df_clean = df[df['product_name'].apply(is_valid_beer_name)].copy()

# Create beer profiles
beer_profiles = df_clean.groupby('product_name').agg({
    'product_review': lambda x: ' '.join(str(rev) for rev in x if pd.notna(rev) and len(str(rev)) > 20),
    'user_rating': 'mean',
    'product_name': 'count'
}).rename(columns={'product_review': 'review_text', 'product_name': 'review_count'})

# Filter beers with sufficient reviews and content
min_reviews = 2
min_text_length = 100
beer_profiles = beer_profiles[
    (beer_profiles['review_count'] >= min_reviews) &
    (beer_profiles['review_text'].str.len() >= min_text_length)
]

print(f"Valid beers found: {len(beer_profiles)}")

if len(beer_profiles) < 10:
    print("Error: Not enough valid beers found")
    exit()

# Select 10 beers
selected_beers = beer_profiles.sample(n=10, random_state=None).index.tolist()
target_beer = selected_beers[0]
comparison_beers = selected_beers[1:]

print(f"Target: {target_beer}")
print(f"Comparing against: {len(comparison_beers)} beers")
print(f"Comparison beers: {comparison_beers[:3]}...") # Show first 3

# Compare review content directly
target_text = beer_profiles.loc[target_beer, 'review_text']
comparison_texts = [beer_profiles.loc[beer, 'review_text'] for beer in comparison_beers]

# Clean text function
def clean_review_text(text):
    if not isinstance(text, str):
        return ""

    # Remove common artifacts
    text = re.sub(r'look smell taste \d+[\d\.\s]*', '', text, flags=re.IGNORECASE)
    text = re.sub(r'character count: \d+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'thanks for the review.*?report', '', text, flags=re.IGNORECASE)
    text = re.sub(r'report\.\.\.', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Clean all texts
target_text_clean = clean_review_text(target_text)
comparison_texts_clean = [clean_review_text(text) for text in comparison_texts]

# Create TF-IDF vectors
all_texts = [target_text_clean] + comparison_texts_clean
vectorizer = TfidfVectorizer(
    max_features=500,
    stop_words='english',
    min_df=1,
    ngram_range=(1, 2)
)

try:
    vectors = vectorizer.fit_transform(all_texts)

    # Calculate similarities
    similarities = cosine_similarity(vectors[0:1], vectors[1:]).flatten()

    # Find most similar
    most_similar_idx = similarities.argmax()
    most_similar_beer = comparison_beers[most_similar_idx]
    highest_similarity = similarities[most_similar_idx]

    print(f"\nMost similar to '{target_beer}':")
    print(f"  {most_similar_beer} (similarity: {highest_similarity:.3f})")

    # Create results with similarities
    results = list(zip(comparison_beers, similarities))
    results.sort(key=lambda x: x[1], reverse=True)

    print(f"\nTop 5 most similar:")
    for i, (beer, sim) in enumerate(results[:5]):
        print(f"  {i+1}. {beer}: {sim:.3f}")

    print(f"\nTask F Complete!")

except Exception as e:
    print(f"Error in similarity calculation: {e}")
    print("Check if review texts contain sufficient content for comparison")

Task F: Direct Beer Content Similarity (Fixed)
Valid beers found: 244
Target: Saison Du Fermier
Comparing against: 9 beers
Comparison beers: ['Imperial Eclipse Stout - Elijah Craig (12 Year)', 'Bourbon Paradise', 'Samuel']...

Most similar to 'Saison Du Fermier':
  Fort Point Pale Ale - Mosaic Dry Hopped (similarity: 0.740)

Top 5 most similar:
  1. Fort Point Pale Ale - Mosaic Dry Hopped: 0.740
  2. Samuel: 0.701
  3. Leaner: 0.648
  4. Heady Topper: 0.433
  5. Imperial Eclipse Stout - Elijah Craig (12 Year): 0.131

Task F Complete!


In Task F, we randomly sampled 10 beers from the dataset. From this set, we selected Saison Du Fermier as the target beer and compared it against the remaining 9.

To perform the comparison, we aggregated all reviews for each beer into a single document, applied text cleaning to remove artifacts (e.g., timestamps, “thanks for the review”), and represented each beer’s text using a TF-IDF vectorization with unigrams and bigrams. We then measured similarity between the target and comparison beers using cosine similarity.

The beer most similar to Saison Du Fermier was Fort Point Pale Ale – Mosaic Dry Hopped (similarity = 0.740). The next closest matches were Samuel (0.701), Leaner (0.648), Heady Topper (0.433), and Imperial Eclipse Stout – Elijah Craig (12 Year) (0.131).

These results are intuitive: Saison Du Fermier and Fort Point Pale Ale – Mosaic Dry Hopped share common descriptors in their reviews around bright, dry, and hop-forward flavor profiles, while Samuel and Leaner also carry overlapping language that emphasizes balance and nuanced fermentation. Beers like Heady Topper appear with lower similarity, reflecting fewer shared descriptors but still some overlap in reviewer language.